In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,AutoModelForSequenceClassification
import torch

In [ ]:
# Configuration : vérifie si l'installation du cuda est correcte
assert torch.cuda.is_available()

use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

folder_name = "../models/flan-t5"
model.save_pretrained(folder_name)

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir='../models/'
)

folder_name = "../models/llama2_7b-4bits"
model.save_pretrained(folder_name)

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir='../models/'
)

folder_name = "../models/mistral_7b-4bits"
model.save_pretrained(folder_name)

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto",cache_dir='../models/')

folder_name = "../models/roberta_hate"
toxicity_model.save_pretrained(folder_name)